$$
\newcommand{\mat}[1]{\boldsymbol {#1}}
\newcommand{\mattr}[1]{\boldsymbol {#1}^\top}
\newcommand{\matinv}[1]{\boldsymbol {#1}^{-1}}
\newcommand{\vec}[1]{\boldsymbol {#1}}
\newcommand{\vectr}[1]{\boldsymbol {#1}^\top}
\newcommand{\rvar}[1]{\mathrm {#1}}
\newcommand{\rvec}[1]{\boldsymbol{\mathrm{#1}}}
\newcommand{\diag}{\mathop{\mathrm {diag}}}
\newcommand{\set}[1]{\mathbb {#1}}
\newcommand{\norm}[1]{\left\lVert#1\right\rVert}
\newcommand{\pderiv}[2]{\frac{\partial #1}{\partial #2}}
\newcommand{\bm}[1]{{\bf #1}}
\newcommand{\bb}[1]{\bm{\mathrm{#1}}}
$$

# Our Project
<a id=part2></a>

imports and stuff

In [17]:
import unittest
import os
import sys
import pathlib
import urllib
import shutil
import re
import zipfile

import numpy as np
import torch
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
test = unittest.TestCase()
plt.rcParams.update({'font.size': 12})
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cpu


### Obtaining the dataset
<a id=part2_1></a>

Getting it from the specific folder
Create a `Dataset` object that will load the extraced images:

OK, let's see what we got. You can run the following block multiple times to display a random subset of images from the dataset.

In [19]:
from hw3.CenterFaceMask import CenterFaceMask, loss_fn
from torch.utils.data import Dataset, DataLoader, random_split
import torch.optim as optim
from hw3.training import CenterFaceMaskTrainer
from hw3.data_loader import ToTensorSmaller, CelebA

# Data
ds = CelebA('./imgs/CelebAMask-HQ/CelebA-HQ-img', 
            #'./imgs/CelebAMask-HQ/CelebA-HQ-img/annotation_indicators.csv',
            './imgs/CelebAMask-HQ/CelebAMask-HQ-mask-anno/CelebAMask-HQ-mask-anno-complete',
            './imgs/CelebAMask-HQ/CelebAMask-HQ-mask-anno/CelebAMask-HQ-mask-anno-complete/sized_and_centers.csv', ToTensorSmaller())
ds_train, ds_test = random_split(ds, [25000, 5000])

dl_train = DataLoader(ds_train, 125, shuffle=True, num_workers=2)
dl_test  = DataLoader(ds_test,  125, shuffle=True, num_workers=2)

# Model
model = CenterFaceMask()

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=2.5e-4)

# Loss
loss = loss_fn

#Trainer
trainer = CenterFaceMaskTrainer(model, loss, (0.1, 1, 1), optimizer, 'cuda')

# Show model and hypers
print(model)

CenterFaceMask(
  (extract_features): DLASeg(
    (base): DLA(
      (base_layer): Sequential(
        (0): Conv2d(3, 16, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), bias=False)
        (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
      (level0): Sequential(
        (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
      (level1): Sequential(
        (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
      (level2): Tree(
        (tree1): BasicBlock(
          (conv1): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(6

In [20]:
import IPython.display

def post_epoch_fn(epoch, train_result, test_result, verbose):
    print(f"epoch num:{epoch}/ntrain result:{train_result}/ntest_result:{test_result}/n")
        
res = trainer.fit(dl_train, dl_test,
                      num_epochs=1, checkpoints='checkpoint_file', early_stopping=15, print_every=2)

--- EPOCH 1/1 ---
train_batch:   0%|                                                                             | 0/200 [00:40<?, ?it/s]


RuntimeError: [enforce fail at ..\c10\core\CPUAllocator.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 19267584000 bytes. Buy new RAM!
